# Boosting

por Mónica Tatiana Gutierrez Ballen

version 1.0, Agosto 2021

This notebook is licensed under a [Creative Commons Attribution-ShareAlike 3.0 Unported License](http://creativecommons.org/licenses/by-sa/3.0/deed.en_US). Special thanks goes to [Rick Muller](http://www.cs.sandia.gov/~rmuller/), Sandia National Laboratories

¿Por qué estamos aprendiendo sobre ensamblaje?

- Método muy popular para mejorar el rendimiento predictivo de los modelos de aprendizaje automático
- Proporciona una base para entender modelos más sofisticados

# Parte 5: Boosting

Aunque el boosting no tiene restricciones algorítmicas, la mayoría de los algoritmos de boosting consisten en aprender iterativamente clasificadores débiles con respecto a una distribución y añadirlos a un clasificador fuerte final. Cuando se añaden, normalmente se ponderan de alguna manera que suele estar relacionada con la precisión de los aprendices débiles. Después de añadir un aprendiz débil, los datos se vuelven a ponderar: los ejemplos mal clasificados ganan peso y los ejemplos correctamente clasificados pierden peso (algunos algoritmos de refuerzo en realidad disminuyen el peso de los ejemplos repetidamente mal clasificados, por ejemplo, el refuerzo por mayoría y BrownBoost). Así, los futuros aprendices débiles se centran más en los ejemplos que los anteriores aprendices débiles clasificaron mal. (Wikipedia)

In [ ]:
from IPython.display import Image
Image(url= "https://www.researchgate.net/publication/351542039/figure/fig1/AS:1022852723662850@1620878501807/Flow-diagram-of-gradient-boosting-machine-learning-method-The-ensemble-classifiers.png", width=900)


## Adaboost

AdaBoost (adaptive boosting) es un algoritmo de aprendizaje de conjunto que puede utilizarse para la clasificación o la regresión. Aunque AdaBoost es más resistente al sobreajuste que muchos algoritmos de aprendizaje automático, suele ser sensible a los datos ruidosos y a los valores atípicos.

AdaBoost se denomina adaptativo porque utiliza múltiples iteraciones para generar un único aprendiz fuerte compuesto. AdaBoost crea el aprendiz fuerte (un clasificador que está bien correlacionado con el clasificador verdadero) añadiendo iterativamente aprendices débiles (un clasificador que está sólo ligeramente correlacionado con el clasificador verdadero). Durante cada ronda de entrenamiento, se añade un nuevo aprendiz débil al conjunto y se ajusta un vector de ponderación para centrarse en los ejemplos que se clasificaron mal en las rondas anteriores. El resultado es un clasificador que tiene mayor precisión que los clasificadores de los aprendices débiles.

Algoritmo:

* Inicializar todos los pesos ($w_i$) a 1 / n/muestras
* Entrenar un clasificador $h_t$ utilizando los pesos
* Estimar el error de entrenamiento $e_t$
* Establecer $alpha_t = log\left(\frac{1-e_t}{e_t}\right)$
* Actualizar los pesos 
$$w_i^{t+1} = w_i^{t}e^{{left(\alpha_t \mathbf{I}\left(y_i \ne h_t(x_t)\right)\$$
* Repetir mientras $e_t<0,5$ y $t<T$


In [ ]:
# read in and prepare the churn data
# Download the dataset
import pandas as pd
import numpy as np

url = 'https://raw.githubusercontent.com/albahnsen/PracticalMachineLearningClass/master/datasets/churn.csv'
data = pd.read_csv(url)

# Create X and y

# Select only the numeric features
X = data.iloc[:, [1,2,6,7,8,9,10]].astype(np.float)
# Convert bools to floats
X = X.join((data.iloc[:, [4,5]] == 'no').astype(np.float))

y = (data.iloc[:, -1] == 'True.').astype(np.int)

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)
n_samples = X_train.shape[0]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  if sys.path[0] == '':
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:16: DeprecationWarning: `np.int` i

In [ ]:
n_estimators = 10
weights = pd.DataFrame(index=X_train.index, columns=list(range(n_estimators)))

In [ ]:
t = 0
weights[t] = 1 / n_samples

Entrenar el clasificador

In [ ]:
from sklearn.tree import DecisionTreeClassifier
trees = []
trees.append(DecisionTreeClassifier(max_depth=1))
trees[t].fit(X_train, y_train, sample_weight=weights[t].values)

DecisionTreeClassifier(max_depth=1)

Error de estimación

In [ ]:
from sklearn.metrics import accuracy_score
y_pred_ = trees[t].predict(X_train)
error = []
error.append(1 - accuracy_score(y_pred_, y_train))
error[t]

0.13613972234661886

In [ ]:
alpha = []
alpha.append(np.log((1 - error[t]) / error[t]))
alpha[t]

1.8477293114995077

Actualizar los pesos

In [ ]:
weights[t + 1] = weights[t]
filter_ = y_pred_ != y_train

In [ ]:
weights.loc[filter_, t + 1] = weights.loc[filter_, t] * np.exp(alpha[t])

Normalizar los pesos

In [ ]:
weights[t + 1] = weights[t + 1] / weights[t + 1].sum()

**Iteración 2 - n_estimadores**

In [ ]:
for t in range(1, n_estimators):
    trees.append(DecisionTreeClassifier(max_depth=1))
    trees[t].fit(X_train, y_train, sample_weight=weights[t].values)
    y_pred_ = trees[t].predict(X_train)
    error.append(1 - accuracy_score(y_pred_, y_train))
    alpha.append(np.log((1 - error[t]) / error[t]))
    weights[t + 1] = weights[t]
    filter_ = y_pred_ != y_train
    weights.loc[filter_, t + 1] = weights.loc[filter_, t] * np.exp(alpha[t])
    weights[t + 1] = weights[t + 1] / weights[t + 1].sum()

In [ ]:
error

[0.13613972234661886,
 0.15629198387819077,
 0.8437080161218092,
 0.8437080161218092,
 0.8437080161218092,
 0.8437080161218092,
 0.8437080161218092,
 0.8437080161218092,
 0.8437080161218092,
 0.8437080161218092]

### Crear clasificación

Sólo los clasificadores cuando el error es < 0,5

In [ ]:
new_n_estimators = np.sum([x<0.5 for x in error])

In [ ]:
y_pred_all = np.zeros((X_test.shape[0], new_n_estimators))
for t in range(new_n_estimators):
    y_pred_all[:, t] = trees[t].predict(X_test)

In [ ]:
y_pred = (np.sum(y_pred_all * alpha[:new_n_estimators], axis=1) >= 1).astype(np.int)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  """Entry point for launching an IPython kernel.


In [ ]:
from sklearn.metrics import f1_score
f1_score(y_pred, y_test.values), accuracy_score(y_pred, y_test.values)

(0.5105105105105104, 0.8518181818181818)

### Uso de sklearn

In [ ]:
from sklearn.ensemble import AdaBoostClassifier

In [ ]:
clf = AdaBoostClassifier()
clf

AdaBoostClassifier()

In [ ]:
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
f1_score(y_pred, y_test.values), accuracy_score(y_pred, y_test.values)

(0.29107981220657275, 0.8627272727272727)

### Gradient Boosting

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier

clf = GradientBoostingClassifier()
clf

GradientBoostingClassifier()

In [ ]:
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
f1_score(y_pred, y_test.values), accuracy_score(y_pred, y_test.values)

(0.5349794238683128, 0.8972727272727272)

## Resumen

In [ ]:
from IPython.display import Image
Image(url= "https://mateusmaiads.github.io/ensemble_qualify/ensemble_methods-2-01.png", width=900)

## Comparación del ensamblaje manual con un enfoque de modelo único

**Ventajas del ensamblaje manual**

- Aumenta la precisión predictiva
- Facilidad de inicio

**Desventajas del ensamblaje manual**

- Disminuye la interpretabilidad
- Se tarda más en entrenar
- Tarda más en predecir
- Es más complejo de automatizar y mantener
- Las pequeñas ganancias de precisión pueden no compensar la complejidad añadida